In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
import time
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [16]:
from datetime import datetime, timedelta
import re

def convert_to_standard_date(date_string):
    date_formats = [
        "%b %d, %Y",
        "%B %d, %Y",
        "%Y-%m-%d",  
    ]
    
    for format_str in date_formats:
        try:
            date_obj = datetime.strptime(date_string, format_str)
            print('Standard Format is : ', date_obj)
            return date_obj.strftime("%Y-%m-%d")
        except ValueError:
            pass
    
    return None  

def is_date_1_week_old(date_string):
    standardized_date = convert_to_standard_date(date_string)
    
    if standardized_date is None:
        print("Date format not recognized.")
        return False
    
    date_obj = datetime.strptime(standardized_date, "%Y-%m-%d")
    print('Standardized date:', date_obj)

    today = datetime.now()

    delta = today - date_obj
    print('Difference of date:', delta)

    if delta <= timedelta(days=7):
        print("true")
        return True
    else:
        return False

# Test with example dates
date_strings = [
    "August 10, 2023",
    "2023-08-05",
    "Invalid date format",
]

for date_str in date_strings:
    print(f"Checking '{date_str}':", is_date_1_week_old(date_str))
    print('\n')


Standard Format is :  2023-08-10 00:00:00
Standardized date: 2023-08-10 00:00:00
Difference of date: 6 days, 16:03:22.166545
true
Checking 'August 10, 2023': True


Standard Format is :  2023-08-05 00:00:00
Standardized date: 2023-08-05 00:00:00
Difference of date: 11 days, 16:03:22.166657
Checking '2023-08-05': False


Date format not recognized.
Checking 'Invalid date format': False




In [17]:
links_list = []
flag = 0
page_number = 1

chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless") 

driver = webdriver.Chrome(options=chrome_options)

while flag < 1:
    driver.get("https://ambcrypto.com/search/Blockchain")
    time.sleep(10)
    wait = WebDriverWait(driver, 5)
        
    try:
        popup_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".corwith-close")))
        popup_button.click()
    except:
        print("No pop-up close button found.")

    content = driver.find_element(By.CLASS_NAME, "archive-posts")
    cards = content.find_elements(By.CLASS_NAME, "home-post")

    article_links = []

    for link_and_card in cards:
        article_link = link_and_card.find_element(By.TAG_NAME, "a").get_attribute("href")
        article_links.append(article_link)  

    for article_link in article_links:
        driver.get(article_link)  
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "post-date")))
        date = driver.find_element(By.CLASS_NAME, "post-date")

        if is_date_1_week_old(date.text):
            print("Raw date text:", date.text)
            links_list.append(article_link)
        else:
            flag = 1
            break    
    page_number += 1
        

No pop-up close button found.
<selenium.webdriver.remote.webelement.WebElement (session="e07cbc6bfb535a3c810dbfb6c266cbe5", element="337FA0EEEF7A7083E08749F5867B68AA_element_29")>
[<selenium.webdriver.remote.webelement.WebElement (session="e07cbc6bfb535a3c810dbfb6c266cbe5", element="337FA0EEEF7A7083E08749F5867B68AA_element_30")>, <selenium.webdriver.remote.webelement.WebElement (session="e07cbc6bfb535a3c810dbfb6c266cbe5", element="337FA0EEEF7A7083E08749F5867B68AA_element_31")>, <selenium.webdriver.remote.webelement.WebElement (session="e07cbc6bfb535a3c810dbfb6c266cbe5", element="337FA0EEEF7A7083E08749F5867B68AA_element_32")>, <selenium.webdriver.remote.webelement.WebElement (session="e07cbc6bfb535a3c810dbfb6c266cbe5", element="337FA0EEEF7A7083E08749F5867B68AA_element_33")>, <selenium.webdriver.remote.webelement.WebElement (session="e07cbc6bfb535a3c810dbfb6c266cbe5", element="337FA0EEEF7A7083E08749F5867B68AA_element_34")>, <selenium.webdriver.remote.webelement.WebElement (session="e07cb

In [18]:
print(len(links_list))
links_list

1


['https://ambcrypto.com/lbank-labs-partners-global-top-university-blockchain-clubs-for-bootcamp/']

In [19]:
list_para = []
para_to_append = str()
count = 1

for link in links_list:
    driver.get(link)
    wait = WebDriverWait(driver, 5)

    try:
        popup_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".corwith-close")))
        popup_button.click()
    except:
        print("No pop-up close button found.")    

    content = driver.find_element(By.CLASS_NAME,"single-post-main-middle")
    paragraph = (content.find_elements(By.TAG_NAME, "p"))
    para_to_append = str()

    for para in paragraph:
        para_to_append += para.text

    list_para.append(para_to_append)
    print(len(links_list) - count, " left")
    count += 1

data_dict = dict()
article_number = 1

for article in list_para:
    if len(article) != 0:
        data_dict["AmbCrypto@Article " + str(article_number)] = article
        article_number += 1

No pop-up close button found.
0  left


In [20]:
import json
with open('AmbCrypto_Crone.json', 'w') as f:
    json.dump(data_dict, f)